Process Example From [Kaggle Notebook](https://www.kaggle.com/hamidhaghshenas/public-score-0-000000)

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
teams = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WTeams.csv')
teams2 = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WTeamSpellings.csv', encoding='latin-1')
season_cresults = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WRegularSeasonCompactResults.csv')
season_dresults = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WRegularSeasonDetailedResults.csv')
tourney_cresults = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WNCAATourneyCompactResults.csv')
tourney_dresults = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WNCAATourneyDetailedResults.csv')
slots = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WNCAATourneySlots.csv')
seeds = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WNCAATourneySeeds.csv')
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
seeds = {**seeds, **{k.replace('2018_', '2019_'): seeds[k] for k in seeds if '2018_' in k}}  # reuse seed 2018 to 2019
cities = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WCities.csv')
gcities = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WGameCities.csv')
seasons = pd.read_csv('womens-machine-learning-competition-2019/WDataFiles/WSeasons.csv')
sub = pd.read_csv('womens-machine-learning-competition-2019/WSampleSubmissionStage2.csv')

In [3]:
teams2 = teams2.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams2.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams2, how='left', on=['TeamID'])

del teams2

teams.head()

,TeamID,TeamName,TeamNameCount
0,3101,Abilene Chr,3
1,3102,Air Force,2
2,3103,Akron,1
3,3104,Alabama,1
4,3105,Alabama A&M,2


In [4]:
season_cresults['ST'] = 'S'
season_dresults['ST'] = 'S'
tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'

games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 3})

In [5]:
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ST
0,2010,11,3103,63,3237,49,2,0,23,54,...,6,10,11,27,11,23,7,6,19,S
1,2010,11,3104,73,3399,68,3,0,26,62,...,14,27,14,26,7,20,4,2,27,S
2,2010,11,3110,71,3224,59,1,0,29,62,...,19,23,17,23,8,15,6,0,15,S
3,2010,11,3111,63,3267,58,1,0,27,52,...,16,25,22,22,15,11,14,5,14,S
4,2010,11,3119,74,3447,70,2,1,30,74,...,11,21,21,32,12,14,4,2,14,S


In [6]:
# some identities of team ids
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']] + sorted([r['WTeamID'], r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'], r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'], r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'], r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LStl,LBlk,LPF,ST,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2
0,2010,11,3103,63,3237,49,2,0,23,54,...,7,6,19,S,2010_3103_3237,3103_3237,3103,3237,2010_3103,2010_3237
1,2010,11,3104,73,3399,68,3,0,26,62,...,4,2,27,S,2010_3104_3399,3104_3399,3104,3399,2010_3104,2010_3399
2,2010,11,3110,71,3224,59,1,0,29,62,...,6,0,15,S,2010_3110_3224,3110_3224,3110,3224,2010_3110,2010_3224
3,2010,11,3111,63,3267,58,1,0,27,52,...,14,5,14,S,2010_3111_3267,3111_3267,3111,3267,2010_3111,2010_3267
4,2010,11,3119,74,3447,70,2,1,30,74,...,4,2,14,S,2010_3119_3447,3119_3447,3119,3447,2010_3119,2010_3447


In [8]:
# get feature of seeds
games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LPF,ST,ID,IDTeams,Team1,Team2,IDTeam1,IDTeam2,Team1Seed,Team2Seed
0,2010,11,3103,63,3237,49,2,0,23,54,...,19,S,2010_3103_3237,3103_3237,3103,3237,2010_3103,2010_3237,0.0,0.0
1,2010,11,3104,73,3399,68,3,0,26,62,...,27,S,2010_3104_3399,3104_3399,3104,3399,2010_3104,2010_3399,0.0,0.0
2,2010,11,3110,71,3224,59,1,0,29,62,...,15,S,2010_3110_3224,3110_3224,3110,3224,2010_3110,2010_3224,0.0,0.0
3,2010,11,3111,63,3267,58,1,0,27,52,...,14,S,2010_3111_3267,3111_3267,3111,3267,2010_3111,2010_3267,0.0,0.0
4,2010,11,3119,74,3447,70,2,1,30,74,...,14,S,2010_3119_3447,3119_3447,3119,3447,2010_3119,2010_3447,0.0,0.0


In [9]:
games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'], r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)  # the label
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] 
games = games.fillna(-1)

games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,Team1,Team2,IDTeam1,IDTeam2,Team1Seed,Team2Seed,ScoreDiff,Pred,ScoreDiffNorm,SeedDiff
0,2010,11,3103,63,3237,49,2,0,23,54,...,3103,3237,2010_3103,2010_3237,0.0,0.0,14,1.0,14,0.0
1,2010,11,3104,73,3399,68,3,0,26,62,...,3104,3399,2010_3104,2010_3399,0.0,0.0,5,1.0,5,0.0
2,2010,11,3110,71,3224,59,1,0,29,62,...,3110,3224,2010_3110,2010_3224,0.0,0.0,12,1.0,12,0.0
3,2010,11,3111,63,3267,58,1,0,27,52,...,3111,3267,2010_3111,2010_3267,0.0,0.0,5,1.0,5,0.0
4,2010,11,3119,74,3447,70,2,1,30,74,...,3119,3447,2010_3119,2010_3447,0.0,0.0,4,1.0,4,0.0


In [11]:
# Competitiveness, includes more game options - overfitting for now in Tournaments
c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF']
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']
gb = games.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

# for now
games = games[games['ST']=='T']

print(games.shape)
games.head()

(567, 47)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,Team1,Team2,IDTeam1,IDTeam2,Team1Seed,Team2Seed,ScoreDiff,Pred,ScoreDiffNorm,SeedDiff
46342,2010,138,3124,69,3201,55,3,0,28,57,...,3124,3201,2010_3124,2010_3201,4.0,13.0,14,1.0,14,-9.0
46343,2010,138,3173,67,3395,66,3,0,23,59,...,3173,3395,2010_3173,2010_3395,8.0,9.0,1,1.0,1,-1.0
46344,2010,138,3181,72,3214,37,2,0,26,57,...,3181,3214,2010_3181,2010_3214,2.0,15.0,35,1.0,35,-13.0
46345,2010,138,3199,75,3256,61,2,0,25,63,...,3199,3256,2010_3199,2010_3256,3.0,14.0,14,1.0,14,-11.0
46346,2010,138,3207,62,3265,42,3,0,24,68,...,3207,3265,2010_3207,2010_3265,5.0,12.0,20,1.0,20,-7.0


In [30]:
gb.head()

,IDTeams_c_score,NumOTsum_c_score,NumOTmean_c_score,NumOTmedian_c_score,NumOTmax_c_score,NumOTmin_c_score,NumOTstd_c_score,NumOTskew_c_score,NumOTnunique_c_score,WFGMsum_c_score,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,3101_3102,0,0.0,0.0,0,0,NaN,NaN,1,24,...,NaN,1,24,24.00,24.0,24,24,NaN,NaN,1
1,3101_3116,0,0.0,0.0,0,0,NaN,NaN,1,25,...,NaN,1,21,21.00,21.0,21,21,NaN,NaN,1
2,3101_3124,0,0.0,0.0,0,0,NaN,NaN,1,30,...,NaN,1,19,19.00,19.0,19,19,NaN,NaN,1
3,3101_3146,0,0.0,0.0,0,0,0.0,0.0,1,191,...,0.320289,6,154,19.25,19.5,22,15,2.12132,-1.0326,6
4,3101_3185,0,0.0,0.0,0,0,NaN,NaN,1,29,...,NaN,1,22,22.00,22.0,22,22,NaN,NaN,1


In [13]:
# submission features

sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub = sub.fillna(-1)

sub.head()

,ID,Pred,WLoc,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,SeedDiff
0,2019_3101_3113,0.5,3,2019,3101,3113,3101_3113,2019_3101,2019_3113,0.0,7.0,-7.0
1,2019_3101_3114,0.5,3,2019,3101,3114,3101_3114,2019_3101,2019_3114,0.0,14.0,-14.0
2,2019_3101_3120,0.5,3,2019,3101,3120,3101_3120,2019_3101,2019_3120,0.0,0.0,0.0
3,2019_3101_3124,0.5,3,2019,3101,3124,3101_3124,2019_3101,2019_3124,0.0,2.0,-2.0
4,2019_3101_3125,0.5,3,2019,3101,3125,3101_3125,2019_3101,2019_3125,0.0,12.0,-12.0


In [14]:
gb.head()

,IDTeams_c_score,NumOTsum_c_score,NumOTmean_c_score,NumOTmedian_c_score,NumOTmax_c_score,NumOTmin_c_score,NumOTstd_c_score,NumOTskew_c_score,NumOTnunique_c_score,WFGMsum_c_score,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,3101_3102,0,0.0,0.0,0,0,NaN,NaN,1,24,...,NaN,1,24,24.00,24.0,24,24,NaN,NaN,1
1,3101_3116,0,0.0,0.0,0,0,NaN,NaN,1,25,...,NaN,1,21,21.00,21.0,21,21,NaN,NaN,1
2,3101_3124,0,0.0,0.0,0,0,NaN,NaN,1,30,...,NaN,1,19,19.00,19.0,19,19,NaN,NaN,1
3,3101_3146,0,0.0,0.0,0,0,0.0,0.0,1,191,...,0.320289,6,154,19.25,19.5,22,15,2.12132,-1.0326,6
4,3101_3185,0,0.0,0.0,0,0,NaN,NaN,1,29,...,NaN,1,22,22.00,22.0,22,22,NaN,NaN,1


In [15]:
games = pd.merge(games, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')  # training data
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')  # test data

In [16]:
# feature columns

col = [c for c in games.columns 
       if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 
                    'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 
                    'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc'] + c_score_col]

print(col)

['Season', 'Team1Seed', 'Team2Seed', 'SeedDiff', 'IDTeams_c_score', 'NumOTsum_c_score', 'NumOTmean_c_score', 'NumOTmedian_c_score', 'NumOTmax_c_score', 'NumOTmin_c_score', 'NumOTstd_c_score', 'NumOTskew_c_score', 'NumOTnunique_c_score', 'WFGMsum_c_score', 'WFGMmean_c_score', 'WFGMmedian_c_score', 'WFGMmax_c_score', 'WFGMmin_c_score', 'WFGMstd_c_score', 'WFGMskew_c_score', 'WFGMnunique_c_score', 'WFGAsum_c_score', 'WFGAmean_c_score', 'WFGAmedian_c_score', 'WFGAmax_c_score', 'WFGAmin_c_score', 'WFGAstd_c_score', 'WFGAskew_c_score', 'WFGAnunique_c_score', 'WFGM3sum_c_score', 'WFGM3mean_c_score', 'WFGM3median_c_score', 'WFGM3max_c_score', 'WFGM3min_c_score', 'WFGM3std_c_score', 'WFGM3skew_c_score', 'WFGM3nunique_c_score', 'WFGA3sum_c_score', 'WFGA3mean_c_score', 'WFGA3median_c_score', 'WFGA3max_c_score', 'WFGA3min_c_score', 'WFGA3std_c_score', 'WFGA3skew_c_score', 'WFGA3nunique_c_score', 'WFTMsum_c_score', 'WFTMmean_c_score', 'WFTMmedian_c_score', 'WFTMmax_c_score', 'WFTMmin_c_score', 'WFT

In [18]:
model = ExtraTreesClassifier(n_estimators=200)
model.fit(games[col].fillna(-1), games['Pred'])
predictions = model.predict(games[col].fillna(-1)).clip(0,1)

print('Log Loss:', log_loss(games['Pred'], predictions))

Log Loss: 9.992007221626415e-16


In [29]:
sub['Pred'] = model.predict_proba(sub[col].fillna(-1))[:, 1]
sub.head()

,ID,Pred,WLoc,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2019_3101_3113,0.510,3,2019,3101,3113,3101_3113,2019_3101,2019_3113,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019_3101_3114,0.600,3,2019,3101,3114,3101_3114,2019_3101,2019_3114,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019_3101_3120,0.470,3,2019,3101,3120,3101_3120,2019_3101,2019_3120,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019_3101_3124,0.625,3,2019,3101,3124,3101_3124,2019_3101,2019_3124,0.0,...,NaN,1.0,19.0,19.0,19.0,19.0,19.0,NaN,NaN,1.0
4,2019_3101_3125,0.560,3,2019,3101,3125,3101_3125,2019_3101,2019_3125,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
sub[["ID", "Pred"]].to_csv("test.csv", index=False)